In [1]:
%cd /kaggle/working
!git clone https://github.com/swapniel99/erav1s13.git
%cd erav1s13
!mkdir -p checkpoints
!git checkout main
!git pull
!ln -s /kaggle/input/pascal-voc-dataset-used-in-yolov3-video/PASCAL_VOC PASCAL_VOC
# !pip install -qr requirements.txt
# !rm -rf lightning_logs
!ls

/kaggle/working
fatal: destination path 'erav1s13' already exists and is not an empty directory.
/kaggle/working/erav1s13
Already on 'main'
Your branch is up to date with 'origin/main'.
Already up to date.
PASCAL_VOC   checkpoints  loss.py    requirements.txt  yolov3.py
README.md    config.py	  model.pth  train.py	       yolov3_train.ipynb
__pycache__  dataset.py   model.py   utils.py


In [2]:
import torch
from model import Model

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

Device Selected: cuda


In [3]:
model = Model()
model.load_state_dict(torch.load('model.pth'))
dataloader = model.predict_dataloader()

In [4]:
import config
from utils import *

In [17]:
check_class_accuracy(model.to('cuda'), dataloader, threshold=0.05)

100%|██████████| 155/155 [01:07<00:00,  2.29it/s]

Class accuracy is: 76.475479%
No obj accuracy is: 98.753448%
Obj accuracy is: 54.959824%


In [20]:
pred_boxes, true_boxes = get_evaluation_bboxes(
                dataloader,
                model,
                iou_threshold=config.NMS_IOU_THRESH,
                anchors=config.ANCHORS,
                threshold=0.05,
            )

100%|██████████| 155/155 [11:41<00:00,  4.53s/it]


In [21]:
mapval = mean_average_precision(
    pred_boxes,
    true_boxes,
    iou_threshold=config.MAP_IOU_THRESH,
    box_format="midpoint",
    num_classes=config.NUM_CLASSES,
)

In [22]:
mapval

tensor(0.2923)